In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
os.chdir('/aichallenge')

In [3]:
import time
t1=time.time()
train = pd.read_table('training_set.tsv')
test = pd.read_table('scoring_set.tsv')
t2=time.time()
print(t2-t1)

/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,16,17,26,40,41,49,53,54,55,100,104,107,109,112,127,128,129,130,133,153,171,220,221,222,225,226,227) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,16,17,26,40,41,49,53,54,55,100,104,107,109,112,127,128,129,130,153,171,220,221,222,225,226,227) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


564.7160155773163


In [4]:
train.columns

Index(['training_data.innovation_challenge_key', 'training_data.renewed_yorn',
       'training_data.instance_id', 'training_data.contract_line_id',
       'training_data.minor_line_yorn', 'training_data.major_line_instance_id',
       'training_data.installation_date',
       'training_data.product_sales_order_type',
       'training_data.product_purchase_order_type',
       'training_data.instance_status',
       ...
       'training_data.service_distributor_base_theater_name',
       'training_data.service_distributor_base_distributor_normalized_name',
       'training_data.contract_line_net_usd_amount',
       'training_data.product_net_price',
       'training_data.sales_node_renewal_rate',
       'training_data.customer_renewal_rate',
       'training_data.partner_renewal_rate',
       'training_data.product_renewal_rate',
       'training_data.service_sales_node_installed_base_sales_node_renewal_rate',
       'training_data.service_partner_installed_base_partner_renewal_rate'],


In [5]:
#remove data set name from columns names to make it uniform
t1=time.time()
columns=[]
for col in train.columns:
    columns.append(col.replace('training_data.',''))
train.columns=columns
columns=[]
for col in test.columns:
    columns.append(col.replace('scoring_set.',''))
test.columns=columns
t2=time.time()
print(t2-t1)

0.001352548599243164


In [6]:
#remove the rows where 'challenge_data.renewed_yorn' is null in training data
t1=time.time()
train=train[train['renewed_yorn'].notnull()]
t2=time.time()
print(t2-t1)

12.986035585403442


In [7]:
#remove the 'challenge_data.renewed_yorn' from training and test data and move it to a seperate variable
t1=time.time()
train_y=train['renewed_yorn']
train = train.drop('renewed_yorn',axis=1)
test_y=test['renewed_yorn']
test = test.drop('renewed_yorn',axis=1)
t2=time.time()
print(t2-t1)

12.154809951782227


In [8]:
#remove columns which have many null values
t1=time.time()
percent = (test.isnull().sum()/test.isnull().count())
test.columns[percent>0.5]
#We would ignore all these 102 columns so we are left with 135 columns
len(test.columns[percent>0.5])
#all_clean_columns=train.columns[percent==0]
#train_clean=train[all_clean_columns]
unclean_columns=test.columns[percent>0.5]
train = train.drop(unclean_columns,axis=1)
test = test.drop(unclean_columns,axis=1)
t2=time.time()
print(t2-t1)

59.33716320991516


In [9]:
len(unclean_columns)

36

In [10]:
unclean_columns

Index(['installation_date', 'service_contract_last_renewal_date',
       'service_contract_classification', 'contract_line_duration_in_days',
       'marketing_program_name', 'service_contract_discount_percentage',
       'pos_bookings_measure_transaction_type', 'pos_adjustment_type',
       'cisco_one_opportunity_type', 'ios_software_version',
       'service_offering', 'service_program', 'service_fee_amount',
       'last_date_of_support', 'end_new_service_attachment_date',
       'end_of_service_contract_renewal_date', 'end_software_maintenance_date',
       'service_level_group', 'mapped_service_list_price',
       'mapped_service_level', 'service_group', 'service_brand_code',
       'service_product_base_service_fee_amount',
       'service_product_base_last_date_of_support',
       'service_product_base_end_new_service_attachment_date',
       'service_product_base_end_of_service_contract_renewal_date',
       'service_product_base_end_software_maintenance_date',
       'service_

In [11]:
#Remove columns which have too many categories and is string type
t1=time.time()
messy_columns = []
for col in train.columns:
    if(train[col].dtype=='O' and len(train[col].unique())>20):
        messy_columns.append(col)
t2=time.time()
print(t2-t1)

62.49336051940918


In [12]:
# 56 messy columns are removed
len(messy_columns)

77

In [13]:
t1=time.time()
train=train.drop(messy_columns,axis=1)
test=test.drop(messy_columns,axis=1)
t2=time.time()
print(t2-t1)

9.921275854110718


In [14]:
train.shape

(6010041, 123)

In [15]:
#Remove the primary key column for data fitting
t1=time.time()
train_X = train.drop('innovation_challenge_key',axis=1)
test_X = test.drop('innovation_challenge_key',axis=1)
t2=time.time()
print(t2-t1)

5.385962963104248


In [16]:
#Number of unique values in each columns
t1=time.time()
catcols=[]
scalcols=[]
for col in train_X.columns:
    if (train_X[col].dtype !='object'):
        if(len(train_X[col].unique())<20):
            catcols.append(col)
        else:
            scalcols.append(col)
t2=time.time()
print(t2-t1)

0.33687829971313477


In [17]:
#handle catcols
t1=time.time()
for col in catcols:
    train_X[col]=train_X[col].astype('category')
    test_X[col]=test_X[col].astype('category')
t2=time.time()
print(t2-t1)

0.6186437606811523


In [22]:
train_X[scalcols]=train_X[scalcols].fillna(train_X[scalcols].median())
test_X[scalcols]=test_X[scalcols].fillna(test_X[scalcols].median())

['contract_line_net_usd_amount',
 'product_net_price',
 'sales_node_renewal_rate',
 'partner_renewal_rate',
 'product_renewal_rate',
 'service_sales_node_installed_base_sales_node_renewal_rate',
 'service_partner_installed_base_partner_renewal_rate']

In [23]:
#Normalize integer data
t1=time.time()
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler = scaler.fit(train_X[scalcols])
train_X[scalcols] = scaler.transform(train_X[scalcols])
scaler = scaler.fit(test_X[scalcols])
test_X[scalcols] = scaler.transform(test_X[scalcols])
t2=time.time()
print(t2-t1)

8.310591459274292


In [24]:
train_X[scalcols].describe()

,contract_line_net_usd_amount,product_net_price,sales_node_renewal_rate,partner_renewal_rate,product_renewal_rate,service_sales_node_installed_base_sales_node_renewal_rate,service_partner_installed_base_partner_renewal_rate
count,6.010041e+06,6.010041e+06,6.010041e+06,6.010041e+06,6.010041e+06,6.010041e+06,6.010041e+06
mean,-3.177913e-17,5.735377e-17,5.787455e-18,-1.343047e-17,2.407080e-18,-9.765462e-19,-3.816333e-18
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.164629e-01,-2.451377e-01,-9.088766e-02,-2.189823e-01,-4.566387e-02,-6.938574e-02,-4.881693e-02
25%,-1.146668e-01,-2.451377e-01,-2.424868e-03,-3.552587e-02,-3.341360e-02,-6.327430e-03,-7.704820e-03
50%,-1.146668e-01,-2.451377e-01,-2.424868e-03,-3.552587e-02,-3.341360e-02,-6.327430e-03,-7.704820e-03
75%,-1.146668e-01,-1.634183e-01,-2.424868e-03,-3.552587e-02,-3.341360e-02,-6.327430e-03,-7.704820e-03
max,2.899812e+02,7.503942e+01,9.262241e+02,4.831607e+02,2.205357e+02,1.934189e+02,7.311580e+02


In [25]:
train_X[catcols].describe()

,product_transaction_type,contract_line_reaction_time_code,sales_hierarchy_level,service_sales_node_base_sales_hierarchy_level
count,6010041,6010041,6010041,6010041
unique,3,4,6,6
top,10002,-1,6,6
freq,5316273,5985929,5248934,5945627


In [26]:
catcols

['product_transaction_type',
 'contract_line_reaction_time_code',
 'sales_hierarchy_level',
 'service_sales_node_base_sales_hierarchy_level']

In [27]:
#convert data to one hot encoding to handle categorical values
t1=time.time()
train_objs_num = len(train_X)
dataset = pd.concat(objs=[train_X, test_X], axis=0)
dataset = pd.get_dummies(dataset)
train_X = dataset[:train_objs_num]
test_X = dataset[train_objs_num:]
t2=time.time()
print(t2-t1)

91.22528433799744


In [28]:
train_X.columns

Index(['service_sales_node_base_sales_hierarchy_level',
       'contract_line_net_usd_amount', 'product_net_price',
       'sales_node_renewal_rate', 'partner_renewal_rate',
       'product_renewal_rate',
       'service_sales_node_installed_base_sales_node_renewal_rate',
       'service_partner_installed_base_partner_renewal_rate',
       'minor_line_yorn_N', 'minor_line_yorn_Y',
       ...
       'service_partner_base_business_owner_ethnicity_code_ASIAN',
       'service_partner_base_business_owner_ethnicity_code_HISPANIC',
       'service_partner_base_business_owner_ethnicity_code_NATIVE',
       'service_partner_base_business_owner_ethnicity_code_SUB',
       'service_partner_base_hubzone_small_business_yorn_N',
       'service_partner_base_hubzone_small_business_yorn_Y',
       'service_partner_base_disabled_veteran_owned_business_yorn_N',
       'service_partner_base_disabled_veteran_owned_business_yorn_Y',
       'distributor_normalized_name_1D5361239B4297AF9198AAEF769C2A4191997

In [29]:
#print the new columns we got
train_X.shape

(6010041, 422)

In [30]:
test_X.shape

(1888195, 422)

In [34]:
test_X.isnull().any().any()

False

In [35]:
#split the data between train and validation set
t1=time.time()
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train_X,train_y,test_size=0.33, random_state=42)
t2=time.time()
print(t2-t1)

22.838346481323242


In [36]:
#Dimensions of training data
X_train.shape

(4026727, 422)

In [37]:
#Lets start fitting different models on the data
#1. start with simple Bayes model
t1=time.time()
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gaussian_model = gnb.fit(X_train,y_train)
t2=time.time()
print(t2-t1)

83.11616110801697


In [38]:
# Lets see the prediction on our validation data
t1=time.time()
pred = gaussian_model.predict_proba(X_valid)
t2=time.time()
print(t2-t1)

17.353989362716675


In [39]:
pred[1,]

array([1.00000000e+00, 2.37830505e-19])

In [40]:
train_y.head()

0    Y
1    Y
2    Y
3    N
4    N
Name: renewed_yorn, dtype: object

In [41]:
from sklearn.metrics import log_loss
log_loss(y_valid,pred[:,1])

8.298727778369816

In [34]:
#find predicted value on test set
pred = gaussian_model.predict_proba(test_X)

In [39]:
#submit the prediction score
submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test['innovation_challenge_key']
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=pred[:,1]

In [40]:
os.chdir('/users/hdpsndbx125')

In [41]:
#very important to sort the values else won't be taken at submission
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')

In [42]:
#persist to a file
submit.to_csv('NaiveBayesGaussian_EntireTraining.csv',index=False)

In [43]:
submit.shape

(1888195, 2)

In [46]:
#fitting a multinomialNB
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=1.0, fit_prior=True)
clf.fit(X_train,y_train)
pred = clf.predict_proba(X_valid)
log_loss(y_valid,pred[:,1])

ValueError: Input X must be non-negative

In [ ]:
#find predicted value on test set
pred = clf.predict_proba(test_X)

In [47]:
from sklearn.naive_bayes import BernoulliNB
mnb = MultinomialNB(alpha=1.0)
mnb.fit(X_train,y_train)
pred = mnb.predict_proba(X_valid)
log_loss(y_valid,pred[:,1])

ValueError: Input X must be non-negative

In [42]:
from sklearn.linear_model import LogisticRegression as lr
t1=time.time()
lr1=lr(max_iter=100)
lr1.fit(X_train,y_train)
pred = lr1.predict_proba(X_valid)
print(log_loss(y_valid,pred[:,1]))
t2=time.time()
print(t2-t1)

0.5493629871845106
3442.311344385147


In [43]:
#find predicted value on test set
pred = lr1.predict_proba(test_X)

In [44]:
#submit the prediction score
submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test['innovation_challenge_key']
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=pred[:,1]
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')
submit.to_csv('LR_100-iter-all-col-train.csv',index=False)

In [45]:
from sklearn.linear_model import SGDClassifier
t1=time.time()
sgd = SGDClassifier(loss="log", penalty="l2",max_iter=100,verbose=True)
sgd.fit(X_train,y_train)
pred = sgd.predict_proba(X_valid)
print(log_loss(y_valid,pred[:,1]))
t2=time.time()
print(t2-t1)

-- Epoch 1
Norm: 4.30, NNZs: 419, Bias: -3.234759, T: 4026727, Avg. loss: 0.956223
Total training time: 8.14 seconds.
-- Epoch 2
Norm: 4.13, NNZs: 419, Bias: -3.198766, T: 8053454, Avg. loss: 0.563116
Total training time: 16.37 seconds.
-- Epoch 3
Norm: 4.08, NNZs: 419, Bias: -3.187869, T: 12080181, Avg. loss: 0.557686
Total training time: 25.55 seconds.
-- Epoch 4
Norm: 4.09, NNZs: 419, Bias: -3.168303, T: 16106908, Avg. loss: 0.555459
Total training time: 34.47 seconds.
-- Epoch 5
Norm: 4.07, NNZs: 419, Bias: -3.161761, T: 20133635, Avg. loss: 0.554265
Total training time: 43.53 seconds.
-- Epoch 6
Norm: 4.06, NNZs: 419, Bias: -3.156190, T: 24160362, Avg. loss: 0.553385
Total training time: 52.66 seconds.
-- Epoch 7
Norm: 4.05, NNZs: 419, Bias: -3.148460, T: 28187089, Avg. loss: 0.552952
Total training time: 61.76 seconds.
-- Epoch 8
Norm: 4.05, NNZs: 419, Bias: -3.143550, T: 32213816, Avg. loss: 0.552558
Total training time: 70.88 seconds.
-- Epoch 9
Norm: 4.04, NNZs: 419, Bias: -3.

Norm: 4.03, NNZs: 419, Bias: -3.053452, T: 273817436, Avg. loss: 0.550369
Total training time: 563.24 seconds.
-- Epoch 69
Norm: 4.03, NNZs: 419, Bias: -3.052706, T: 277844163, Avg. loss: 0.550350
Total training time: 571.06 seconds.
-- Epoch 70
Norm: 4.03, NNZs: 419, Bias: -3.052595, T: 281870890, Avg. loss: 0.550349
Total training time: 578.88 seconds.
-- Epoch 71
Norm: 4.03, NNZs: 419, Bias: -3.051227, T: 285897617, Avg. loss: 0.550364
Total training time: 586.65 seconds.
-- Epoch 72
Norm: 4.03, NNZs: 419, Bias: -3.051159, T: 289924344, Avg. loss: 0.550352
Total training time: 594.64 seconds.
-- Epoch 73
Norm: 4.03, NNZs: 419, Bias: -3.050775, T: 293951071, Avg. loss: 0.550344
Total training time: 602.53 seconds.
-- Epoch 74
Norm: 4.03, NNZs: 419, Bias: -3.049632, T: 297977798, Avg. loss: 0.550339
Total training time: 610.57 seconds.
-- Epoch 75
Norm: 4.03, NNZs: 419, Bias: -3.049308, T: 302004525, Avg. loss: 0.550344
Total training time: 618.47 seconds.
-- Epoch 76
Norm: 4.03, NNZs

In [46]:
print(log_loss(y_valid,pred[:,1]))

0.5499836427475958


In [49]:
#submit the prediction score
submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test['innovation_challenge_key']
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=sgd.predict_proba(test_X)[:,1]
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')
submit.to_csv('SGD_100iter_all_cols_train.csv',index=False)

In [50]:
#trying Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
t1=time.time()
rfc = RandomForestClassifier(max_depth=4, random_state=0)
rfc.fit(X_train, y_train)
pred = rfc.predict_proba(X_valid)
print(log_loss(y_valid,pred[:,1]))
t2=time.time()
print(t2-t1)

0.5660281905835715
74.95241832733154


In [51]:
pred = rfc.predict_proba(test_X)

In [52]:
#submit the prediction score
submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test['innovation_challenge_key']
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=pred[:,1]
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')
submit.to_csv('RFC_depth4_all_cols_train.csv',index=False)

In [ ]:
from sklearn import svm
import time
t1=time.time()
svc= svm.SVC(kernel='rbf',probability=True,verbose=True)
svc.fit(X_train, y_train)
t2=time.time()
pred = svc.predict_proba(X_valid)
log_loss(y_valid,pred[:,1])

[LibSVM]

In [1]:
log_loss(y_valid,pred[:,1])

NameError: name 'log_loss' is not defined

In [54]:
clean_col_name=[]
for col in X_train.columns:
    col = col.replace('<','X')
    col = col.replace('>','Y')
    col = col.replace('[','A')
    col = col.replace(']','B')
    clean_col_name.append(col)
X_train.columns = clean_col_name
clean_col_name=[]
for col in X_valid.columns:
    col = col.replace('<','X')
    col = col.replace('>','Y')
    col = col.replace('[','A')
    col = col.replace(']','B')
    clean_col_name.append(col)
X_valid.columns = clean_col_name

In [56]:
import gc
del dataset
gc.collect()

1258

In [ ]:
#Time for xgboost
import xgboost as xgb
import time
t1 = time.time()
model1 = xgb.XGBClassifier(verbose=True)
#model2 = xgb.XGBClassifier(n_estimators=100, max_depth=8, learning_rate=0.1, subsample=0.5)
xgb1 = model1.fit(X_train, y_train)
#xgb2 = model2.fit(X_train.iloc[0:10,], y_train[0:10])
t2 = time.time()
t2-t1

In [50]:
t2-t1

148.6567051410675

In [51]:
pred=xgb1.predict_proba(X_valid)

In [52]:
from sklearn.metrics import log_loss
log_loss(y_valid,pred[:,1])

0.5479801843282507

In [53]:
#submit the prediction score
submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test['innovation_challenge_key']
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=xgb1.predict_proba(test_X)[:,1]
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')
submit.to_csv('XGB-entiretrainscl-allcol.csv',index=False)

In [57]:
os.listdir('.')

['XGB-entiretrainscl-allcol.csv',
 'LR_100iter.csv',
 'scoring_set.tsv',
 'LR_100-iter-all.csv',
 'training_set.tsv',
 'challengeData.tsv',
 'lost+found']

In [30]:
#Trying MLPerceptron
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(20,20,20))

In [33]:
t1=time.time()
mlp.fit(X_train,y_train)
t2=time.time()
print(t2-t1)

MemoryError: 

In [43]:
from sklearn.metrics import log_loss
pred = mlp.predict_proba(X_valid)
log_loss(y_valid,pred[:,1])

0.4238643172502673

In [33]:
#submit the prediction score
submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test['innovation_challenge_key']

In [35]:
pred=mlp.predict_proba(test_X)

In [37]:
len(pred)

1888195

In [38]:
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=pred[:,1]
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')
submit.to_csv('MLP_20_20_20_scl.csv',index=False)